# Day 1

## Part 1
Figure out the most calories carried by any elf.

In [ ]:
; TODO stream process line by by line rather than read in the entire file into memory
(let ((lines (uiop:read-file-lines "input_day1.txt"))
      ; temporary accumulator for each elfs calories
      (cal_ct 0)
      ; variable to track the maximum calories per elf
      (max_cal 0))
    (dolist (line lines)
        ; elves are demarcated by empty lines
        (if (string= "" line)
            (progn (when (> cal_ct max_cal) (setq max_cal cal_ct)) (setq cal_ct 0))
            (incf cal_ct (parse-integer line))
        )
    )
 ; return the maximum number of calories found on an elf
 max_cal)
    

## Part 2

Find the sum of the calories for the top 3 elves. 

In [ ]:
(let (
      (lines (uiop:read-file-lines "input_day1.txt"))
      (cal_ct 0)
      (calories nil))
    (dolist (line lines)
        (if (string= "" line)
            (progn (push cal_ct calories) (setq cal_ct 0))
            (progn (incf cal_ct (parse-integer line)))
        )
    )
    (let ((sorted (sort calories #'>)))
         (+ (first sorted) (second sorted) (third sorted)))
)


# Day 2

## Part 1

Score games of rock-paper-scissors. We'll try first with a brute force if-else chain.

In [ ]:
(defun score_selected_shape (b)
     (if (string= b "X")
         ; 1 point for playing rock
         1
         (if (string= b "Y")
             ; 2 points for playing paper
             2
             (if (string= b "Z")
                 ; 3 points for playing scissors
                 3))))

(defun score_rock_paper_scissors (a b)
    ; they played rock
    (if (string= a "A")
        ; we played rock -> tie
        (if (string= b "X")
            3
            ; we played paper -> we won
            (if (string= b "Y")
                6
                ; we played scissors -> we lost
                (if (string= b "Z")
                    0)))
    ; they played paper
    (if (string= a "B")
        ; we played rock -> we lose
        (if (string= b "X")
            0
            ; we played paper -> tie
            (if (string= b "Y")
                3
                ; we played scissors -> we win
                (if (string= b "Z")
                    6)))
    ; they played scissors
    (if (string= a "C")
        ; we played rock -> we win
        (if (string= b "X")
            6
            ; we played paper -> we lose
            (if (string= b "Y")
                0
                ; we played scissors -> tie
                (if (string= b "Z")
                    3)))))))

(defun score_game (a b)
    "Score a single game of rock-paper-scissors. Opponent plays `a` from {A,B,C}` we play `b` from {X,Y,Z}"
    (+ (score_selected_shape b) (score_rock_paper_scissors a b)))
    
(let ((lines (uiop:read-file-lines "input_day2.txt")))
     (loop for line in lines sum (score_game (string (char line 0)) (string (char line 2)))))

Now let's try association lists.

In [ ]:
(let ((lines (uiop:read-file-lines "input_day2.txt"))
      (shape_score '(("X" . 1)("Y" . 2) ("Z" . 3)))
      (game_score '(("A X" . 3)
                    ("A Y" . 6)
                    ("A Z" . 0)
                    ("B X" . 0)
                    ("B Y" . 3)
                    ("B Z" . 6)
                    ("C X" . 6)
                    ("C Y" . 0)
                    ("C Z" . 3))))
    (loop for line in lines
          for tmp_shape_score = (cdr (assoc (char line 2) shape_score :test #'string=))
          for tmp_game_score = (cdr (assoc line game_score :test #'string=))
          sum (+ tmp_shape_score tmp_game_score)))

## Part 2

Score games of rock paper scissors again.

In [ ]:
(let ((lines (uiop:read-file-lines "input_day2.txt"))
      (shape_score '(("X" . 1)("Y" . 2) ("Z" . 3)))
      (game_score '(("A X" . 3)
                    ("A Y" . 6)
                    ("A Z" . 0)
                    ("B X" . 0)
                    ("B Y" . 3)
                    ("B Z" . 6)
                    ("C X" . 6)
                    ("C Y" . 0)
                    ("C Z" . 3)))
      (what_play  '(("A X" . "Z") ;; they chose rock, we need to lose so choose scissors
                    ("A Y" . "X")
                    ("A Z" . "Y")
                    ("B X" . "X") ;; they chose paper, we need to lose so choose rock
                    ("B Y" . "Y")
                    ("B Z" . "Z")
                    ("C X" . "Y") ;; they chose scissors, we need to lose so choose paper
                    ("C Y" . "Z")
                    ("C Z" . "X"))))
    (loop for line in lines
          for played = (cdr (assoc line what_play :test #'string=))
          for tmp_shape_score = (cdr (assoc played shape_score :test #'string=))
          for new_line = (replace line played :start1 2)
          for tmp_game_score = (cdr (assoc new_line game_score :test #'string=))
          sum (+ tmp_shape_score tmp_game_score)))


# Day 3

## Part 1

Sum of priorities of items in both compartments.

In [ ]:
;; alist mapping characters to integer priority
(defparameter priorities (concatenate 'list
                   (loop for i from 1 to 26 collect (cons (code-char (+ 96 i)) i))
                   (loop for i from 1 to 26 collect (cons (code-char (+ 64 i)) (+ 26 i)))))

(defun find_item_in_both (contents)
    "Find character that is in both the first half and second half of the `contents` string."
    (let ((first_half (subseq contents 0 (/ (length contents) 2)))
          (second_half (subseq contents (/ (length contents) 2))))
         ;; assume there is only one so can take first item in intersection of characters
         (first (intersection (coerce first_half 'list) (coerce second_half 'list))))
)

(let ((lines (uiop:read-file-lines "input_day3.txt")))
     (loop for line in lines
           sum (cdr (assoc (find_item_in_both line) priorities)))
)

In [ ]:
(let ((lines (uiop:read-file-lines "input_day3.txt")))
    ;; step by groups of 3
     (loop for x upto (- (/ (length lines) 3) 1)
           for group = (subseq lines (* 3 x) (+ (* 3 x) 3))
           ;; do (print (reduce #'intersection (mapcar (lambda (x) (coerce x 'list)) group))))
           sum (cdr (assoc (first (reduce #'intersection (mapcar (lambda (x) (coerce x 'list)) group))) priorities)))
)

`intersection` assumes set lists where "A set doesn’t contain twice the same element and is unordered". ([cookbook intersection of lists](https://lispcookbook.github.io/cl-cookbook/data-structures.html#intersection-of-lists)). If we have repeated elements in the list then we get multiple results


In [ ]:
(reduce #'intersection (mapcar (lambda (x) (coerce x 'list)) (list "hlLwDGLNnTrbNdtbdb" "ZfsvJbsJTrsttHTR" "vjPQJPFcmZjVFZFvbbVcJZJPBBhLMnDBznnGMDLMwBhlmBBl")))

# Day 4

## Part 1

Calculate number of complete overlaps.

In [ ]:
(ql:quickload "cl-ppcre")

In [ ]:
(let ((lines (uiop:read-file-lines "input_day4.txt")))
     (loop for line in lines
           count (ppcre:register-groups-bind ((#'parse-integer a b c d))
            ("(\\d+)-(\\d+),(\\d+)-(\\d+)" line)
            ;; assumes ordered a,b and c,d
            (or (and (>= a c) (<= b d)) (and (>= c a) (<= d b))))
    )
)

In [ ]:
(let ((lines (uiop:read-file-lines "input_day4.txt")))
     (loop for line in lines
           count (ppcre:register-groups-bind ((#'parse-integer a b c d))
            ("(\\d+)-(\\d+),(\\d+)-(\\d+)" line)
            ;; check whether any limit lies within the other bounds
            (or (and (>= a c) (<= a d)) (and (>= b c) (<= b d)) (and (>= c a) (<= c b)) (and (>= d a) (<= d b))))
    )
)

# Day 5

## Part 1

Rearange crates. Puzzle input:

```
    [H]         [D]     [P]        
[W] [B]         [C] [Z] [D]        
[T] [J]     [T] [J] [D] [J]        
[H] [Z]     [H] [H] [W] [S]     [M]
[P] [F] [R] [P] [Z] [F] [W]     [F]
[J] [V] [T] [N] [F] [G] [Z] [S] [S]
[C] [R] [P] [S] [V] [M] [V] [D] [Z]
[F] [G] [H] [Z] [N] [P] [M] [N] [D]
 1   2   3   4   5   6   7   8   9 
```

In [ ]:
(defparameter crates (list
                      (list "W" "T" "H" "P" "J" "C" "F" )
                      (list "H" "B" "J" "Z" "F" "V" "R" "G")
                      (list "R" "T" "P" "H")
                      (list "T" "H" "P" "N" "S" "Z")
                      (list "D" "C" "J" "H" "Z" "F" "V" "N")
                      (list "Z" "D" "W" "F" "G" "M" "P")
                      (list "P" "D" "J" "S" "W" "Z" "V" "M")
                      (list "S" "D" "N")
                      (list "M" "F" "S" "Z" "D")))

(let ((lines (uiop:read-file-lines "input_day5.txt")))
     (loop for line in lines
           do (ppcre:register-groups-bind ((#'parse-integer to_move from_stack to_stack))
            ("move (\\d+) from (\\d+) to (\\d+)" line)
            (let ((tmp_stack nil))
                 (progn
                     (loop repeat to_move do (push (pop (elt crates (1- from_stack))) tmp_stack))
                     (loop repeat to_move do (push (pop tmp_stack) (elt crates (1- to_stack))))
                )))))
(loop for stack in crates do (format t "~a" (first stack)))

# Day 6

## Part 1

Find the start of frame marker with 4 unique characters. 

In [ ]:
(defparameter datastream (uiop:read-file-string "input_day6.txt"))
;; (defparameter datastream "mjqjpqmgbljsphdztnvjfqwrcgsmlb")

In [ ]:
(loop for i from 4 to (1- (length datastream))
      when (> (length (remove-duplicates (subseq datastream (- i 4) i) )) 3) return i)

## Part 2

Start of message with 14 unique characters. Let's make a more generic function.

In [ ]:
(defun find_sof (ds len)
    (loop for i from len to (1- (length ds))
      when (> (length (remove-duplicates (subseq ds (- i len) i) )) (1- len)) return i))
    

In [ ]:
(find_sof datastream 14)

# Day 7

## Part 1

Walk a directory structure given a series of terminal inputs and outputs. We'll need a few more pieces here:
1. a tree structure with `folder` and `file` objects; the tree could be a recursive nested list of lists where the nodes are directories and the files are leaf nodes
2. a `folder` has a `name`, `parent` and `children`
3. a `file` has a `name`, `parent`, and `size` 
4. a state machine to parse the input and build the tree

In [13]:
(defclass folder ()
    ((name :initarg :name :accessor name)
     (parent :initarg :parent :initform nil :accessor parent)
     (children :initform nil :accessor children)))

(defclass file ()
    ((name :initarg :name :accessor name)
     (parent :initarg :parent :accessor parent)
     (size :initarg :size :accessor size)))

;; add a child to a folder
(defmethod add-child ((target folder) (child file))
    (setf (parent child) target)
    (push child (children target)))
(defmethod add-child ((target folder) (child folder))
    (setf (parent child) target)
    (push child (children target)))

;; get child by name
(defmethod get-child-by-name ((target folder) (child-name string))
    (find-if #'(lambda (x) (string= (name x) child-name)) (children target)))

;; directory size
(defmethod size ((dir folder))
    (reduce '+ (children dir) :key (lambda (x) (size x))))

#<STANDARD-CLASS COMMON-LISP-USER::FOLDER>

#<STANDARD-CLASS COMMON-LISP-USER::FILE>

#<STANDARD-METHOD COMMON-LISP-USER::ADD-CHILD (FOLDER FILE) {70052AA083}>

#<STANDARD-METHOD COMMON-LISP-USER::ADD-CHILD (FOLDER FOLDER) {7005337AA3}>

#<STANDARD-METHOD COMMON-LISP-USER::GET-CHILD-BY-NAME (FOLDER STRING) {70053F1863}>

#<STANDARD-METHOD COMMON-LISP-USER::SIZE (FOLDER) {7005454E03}>

SB-KERNEL:REDEFINITION-WITH-DEFMETHOD: redefining ADD-CHILD (#<STANDARD-CLASS COMMON-LISP-USER::FOLDER>
                                                             #<STANDARD-CLASS COMMON-LISP-USER::FILE>) in DEFMETHOD
SB-KERNEL:REDEFINITION-WITH-DEFMETHOD: redefining ADD-CHILD (#<STANDARD-CLASS COMMON-LISP-USER::FOLDER>
                                                             #<STANDARD-CLASS COMMON-LISP-USER::FOLDER>) in DEFMETHOD
SB-KERNEL:REDEFINITION-WITH-DEFMETHOD: redefining GET-CHILD-BY-NAME (#<STANDARD-CLASS COMMON-LISP-USER::FOLDER>
                                                                     #<BUILT-IN-CLASS COMMON-LISP:STRING>) in DEFMETHOD
SB-KERNEL:REDEFINITION-WITH-DEFMETHOD: redefining SIZE (#<STANDARD-CLASS COMMON-LISP-USER::FOLDER>) in DEFMETHOD


In [2]:
(ql:quickload "cl-ppcre")

To load "cl-ppcre":
  Load 1 ASDF system:
    cl-ppcre
; Loading "cl-ppcre"
..

("cl-ppcre")

In [3]:
(defclass terminal-parser ()
    ((current-directory :initform nil :accessor pwd)))

(defmethod parse-line ((tp terminal-parser) (line string))
    (cond
        ;; list the folder contents - nothing for us to do but would be nice to track state
        ((string= line "$ ls")
         ;; (format t "~&Listing folder contents")
         )
        ;; change directory commands
        ((ppcre:register-groups-bind (cd_to) ("\\$ cd ([\\w\\/\\.]+)" line)
            (cond 
                ((string= cd_to "/")
                     ;; (format t "~&Moving to root")
                     (setf (pwd tp) root))
                ((string= cd_to "..")
                     ;; (format t "~&Moving up to parent")
                     (setf (pwd tp) (parent (pwd tp))))
                (t
                     ;; (format t "~&Moving to folder ~a" cd_to)
                     (setf (pwd tp) (get-child-by-name (pwd tp) cd_to))))))
        ;; a directory listing
        ((ppcre:register-groups-bind (folder_name) ("dir (\\w+)" line)
             ;; (format t "~&Adding folder named ~a to children of ~a" folder_name (name (pwd tp)))
             (add-child (pwd tp) (make-instance 'folder :name folder_name))))         
        
        ;; a file listing
        ((ppcre:register-groups-bind (file_size file_name) ("(\\d+) ([\\w\\.]+)" line)
             ;; (format t "~&Adding file named ~a with size ~a to children of ~a" file_name file_size (name (pwd tp)))
             (add-child (pwd tp) (make-instance 'file :name file_name :size (parse-integer file_size)))))         
        
    ))
            

#<STANDARD-CLASS COMMON-LISP-USER::TERMINAL-PARSER>

#<STANDARD-METHOD COMMON-LISP-USER::PARSE-LINE (TERMINAL-PARSER STRING) {70089EBFC3}>

SB-INT:TYPE-STYLE-WARNING: This is not a STRING:
 NIL
See also:
  The SBCL Manual, Node "Handling of Types"
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::ROOT


In [11]:
(defparameter tp (make-instance 'terminal-parser))
(defparameter root (make-instance 'folder :name "root"))

(let ((lines (uiop:read-file-lines "input_day7.txt")))
     (loop for line in lines do (parse-line tp line)))

TP

ROOT

NIL

Recursion based approach. Since the `size` method is already recursive the rewalks the tree many times.  

In [15]:
;; depth-first walk of the file system
(defun folder-sizes (pwd)
    (labels ((rec (pwd acc)
        (loop :for item :in (children pwd)
              :when (typep item 'folder)
              :do 
                  ;; (format t "~& dir name ~a; item name: ~a" (name pwd) (name item))
                  (setf acc (append acc (rec item nil)))
                  (push (size item) acc)
                  ;; (format t "~& acc: ~a" acc) 
              :finally (return acc))))
    (rec root nil)))
(let ((sizes (folder-sizes root)))
     (loop for s in sizes
          :when (< s 100000) :sum s))

FOLDER-SIZES

1989474

SB-INT:SIMPLE-STYLE-WARNING: The variable PWD is defined but never used.
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::FOLDER-SIZES in DEFUN


Queue based approach

In [26]:
(let ((stack (remove-if-not (lambda (x) (typep x 'folder)) (children root)))
      (sizes nil))
    (loop :while (> (length stack) 0)
        do
        (push (size (first stack)) sizes)
        (setf stack (append stack (remove-if-not (lambda (x) (typep x 'folder)) (children (first stack)))))
        (pop stack))
     
    (loop for s in sizes
          :when (< s 100000) :sum s))

1989474

## Part 2

Find which directory to delete to free up necessary space.

In [33]:
(let ((sizes (folder-sizes root))
      (disk-size 70000000)
      (space-needed 30000000))
     ;; find out how much we need to delete
     (let ((min-delete (- space-needed (- disk-size (size root)))))
     ;; should defend against not needing to delete anything
     (print min-delete)
     (loop for s in (sort sizes #'<)
           :when (> s min-delete)
           :return s)))

1111607


1072511 